5-7. 미니 프로젝트 : 가위바위보 분류기를 만들자 


<<목차>>

1)데이터를 준비하자

2)딥러닝 네트워크 설계하기

3)딥러닝 네트워크 학습시키기

4)얼마나 잘 만들었는지 확인하기(테스트)

5)더 좋은 네트워크 만들어보기

6)루브릭

7)회고

1)데이터를 준비하자

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image
import glob
import os
from tensorflow import keras
import matplotlib.pyplot as plt

print(tf.__version__)
print(np.__version__)

2022-10-07 23:50:55.687265: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-07 23:50:55.845986: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-07 23:50:56.408353: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.5/lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib64
2022-10-07 23:50:56.408413: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not l

2.10.0
1.23.3


In [2]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
	
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/rock"
resize_images(image_dir_path)

100  images to be resized.
100  images resized.


In [3]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/scissor"
resize_images(image_dir_path)

100  images to be resized.
100  images resized.


In [4]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/paper"
resize_images(image_dir_path)

94  images to be resized.
94  images resized.


이미지들을 모두 28x28으로 리사이즈

In [5]:
import numpy as np

def load_data(img_path, number_of_data=300):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1

    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

2)딥러닝 네트워크 설계하기

In [6]:
image_dir_path = "/home/june/다운로드/RCP"
(x_train, y_train)=load_data(image_dir_path)

x_train_norm = x_train/255.0

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 294 입니다.
x_train shape: (300, 28, 28, 3)
y_train shape: (300,)


In [7]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(256, (3, 3), activation = 'relu', input_shape=(28, 28, 3)))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Conv2D(512, (3, 3), activation = 'relu'))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Conv2D(512, (3, 3), activation = 'relu'))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation = 'relu'))
model.add(keras.layers.Dense(3, activation = 'softmax'))

2022-10-07 23:51:10.177393: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 23:51:10.197303: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 23:51:10.197459: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 23:51:10.197942: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 256)       7168      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 256)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 512)       1180160   
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 512)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 512)         2359808   
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 1, 512)        0

3)딥러닝 네트워크 학습시키기

In [9]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train_norm, y_train, epochs = 10)

Epoch 1/10


2022-10-07 23:51:19.702879: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8300
2022-10-07 23:51:20.711699: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


10/10 [==============================] - 3s 30ms/step - loss: 1.1401 - accuracy: 0.3067
Epoch 2/10
10/10 [==============================] - 0s 11ms/step - loss: 1.1027 - accuracy: 0.3033
Epoch 3/10
10/10 [==============================] - 0s 10ms/step - loss: 1.1027 - accuracy: 0.3533
Epoch 4/10
10/10 [==============================] - 0s 10ms/step - loss: 1.0914 - accuracy: 0.3533
Epoch 5/10
10/10 [==============================] - 0s 10ms/step - loss: 1.0797 - accuracy: 0.3533
Epoch 6/10
10/10 [==============================] - 0s 10ms/step - loss: 1.0769 - accuracy: 0.3533
Epoch 7/10
10/10 [==============================] - 0s 9ms/step - loss: 1.0765 - accuracy: 0.3533
Epoch 8/10
10/10 [==============================] - 0s 9ms/step - loss: 1.0767 - accuracy: 0.3500
Epoch 9/10
10/10 [==============================] - 0s 9ms/step - loss: 1.0765 - accuracy: 0.3033
Epoch 10/10
10/10 [==============================] - 0s 9ms/step - loss: 1.0765 - accuracy: 0.3533


4)얼마나 잘 만들었는지 확인하기(테스트)

In [10]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/test/rock"
resize_images(image_dir_path)

100  images to be resized.
100  images resized.


In [11]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/test/scissor"
resize_images(image_dir_path)

100  images to be resized.
100  images resized.


In [12]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/test/paper"
resize_images(image_dir_path)

100  images to be resized.
100  images resized.


In [13]:
image_dir_path = "/home/june/다운로드/RCP/test"
(x_test, y_test)=load_data(image_dir_path, 300)

x_test_norm = x_test/255.0

학습데이터(x_train)의 이미지 개수는 300 입니다.


In [14]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - loss: 1.9225 - accuracy: 0.3333 - 153ms/epoch - 15ms/step
test_loss: 1.9224752187728882 
test_accuracy: 0.3333333432674408


정확도는 낮은 편이다. 다른 모델들을 사용해봐야 할것이다.

5)더 좋은 네트워크 만들어보기

6) 루브릭


1. 이미지 분류기 모델이 성공적으로 만들어졌는가?

학습과정이 정상적으로 수행되었으며, 학습 결과에 대한 그래프를 시각화(ex. train acc / train loss / val acc / val loss 등) 해 보았음

2. 오버피팅을 극복하기 위한 적절한 시도가 있었는가?

오버피팅 극복을 위하여 데이터셋의 다양성, 정규화 등을 2가지 이상 시도해보았음

3. 분류모델의 test accuracy가 기준 이상 높게 나왔는가?

60% 이상 도달하였음
